In [1749]:
#Suzan Iloglu, May 21,2020
import gurobipy as gp
from gurobipy import GRB
from itertools import product
import pandas as pd
import numpy as np


In [3]:
from IPython.display import Image

![Contact Tracing Allocation](image1.png)

## Contact Tracing Coverage Model

### Objective and Prerequisites

In this model, we solve capacitated coverage model: how to create contact tracer centers to provide coverage for the people who need to be traced with the goal of maximizing the cumulative weighted total number of people traced. We implement this model in the Gurobi Python interface and compute optimal solution

### Motivation


### Problem Description


### Model Formulation 
 
#### Sets
$I $ set of contact tracing demand points (census tracts)<br>
$J$  set of contact tracer center locations (counties)<br>
$S$  set of states<br>
$N_i$ set of contact tracer center locations that can serve demand point $i \in I$<br>
$M_j$ set of contact tracer center locations that belongs to state $s \in S$<br>

#### Parameters
$w_{i}$  weight assigned to demand point $i \in I$ <br>
$d_{i}$  total number of contact tracer needed at demand point $i \in I$<br>
$z_{j}$  weight assigned to contact tracer location  $j \in J$<br>
$c_{s}$  total number of contact tracer capacity at each state $s \in S$<br>


The decision variables for the model are as follows: <br>
 $y_{j} = $the number of contact tracer at center $j \in J$. <br>
 $x_{ij} =$ the number of community health care workers assigned to demand point $i \in I$ by contact tracing center $j \in J$, and 0 otherwise. <br>



The integer programming formulation of our model is as follows.
<br>
$$
\begin{align}
    \max & \sum_{j \in J} \sum_{i \in I} w_{i}x_{ij} + \sum_{j \in J} z_{j}y_{j} && \\
    \text{s.t. } &\sum_{j \in N_{i}}  x_{ij} \leq  d_i &\text{ for }& i \in I  \\
    & \sum_{i \in I: j \in N_{i}} x_{ij} \leq y_{j} \ &\text{ for }& j \in J \\
    & \sum_{j \in M_s}  y_{j} \leq c_s &\text{ for }& s \in S  \\
    & y_{j} \leq d_j &\text{ for }& j \in J  \\
    & x_{ij}  \text{ integer }&\text{  for }& i\in I, j \in J \\
    & y_{j}  \text{ integer }&  \text{ for } &j \in J 
\end{align}
$$

In [1750]:
df_1 = pd.read_csv('County_based_demand.csv')

In [1751]:
TextFileReader = pd.read_csv('svi_2018_tracts_state_ranked.csv',chunksize=5000)
df_svi_x = []
for df in TextFileReader:
    df_svi_x.append(df)

df_svi = pd.concat(df_svi_x, sort=False)

df_svi.head(5)
df_svi.shape

#print ('merged',df.count)
#print ('before merged', df_1.count)

(72837, 11)

In [1752]:
df = pd.merge(left = df_1, right = df_svi, how = 'right', right_on = 'STCNTY', left_on = 'FIPS' )

#df.to_dict('list')
df['RPL_ThemesStates'] = df['RPL_ThemesStates'].fillna(0)

#df.type


df.head(10)
#df.shape

,FIPS_x,County_Name,State,Population,14-Day_Case_Load,County_Baseline_CT_Need,Total_COVID_Need,Final_Need,Unique_Number,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS_y,AREA_SQMI,E_TOTPOP,M_TOTPOP,RPL_ThemesStates
0,1001,Autauga,AL,55869,45,9,23,23,45801,1,ALABAMA,AL,1001,Autauga,1001020200,1.284051,2028,192,0.617547
1,1001,Autauga,AL,55869,45,9,23,23,45805,1,ALABAMA,AL,1001,Autauga,1001021000,149.369459,2550,239,0.580068
2,1001,Autauga,AL,55869,45,9,23,23,609,1,ALABAMA,AL,1001,Autauga,1001020100,3.790677,1923,253,0.155026
3,1001,Autauga,AL,55869,45,9,23,23,613,1,ALABAMA,AL,1001,Autauga,1001020801,47.981925,2826,324,0.121806
4,1001,Autauga,AL,55869,45,9,23,23,45806,1,ALABAMA,AL,1001,Autauga,1001021100,184.615348,2945,335,0.706985
5,1001,Autauga,AL,55869,45,9,23,23,611,1,ALABAMA,AL,1001,Autauga,1001020400,2.464982,3831,337,0.097104
6,1001,Autauga,AL,55869,45,9,23,23,45802,1,ALABAMA,AL,1001,Autauga,1001020600,3.104879,3705,342,0.465928
7,1001,Autauga,AL,55869,45,9,23,23,610,1,ALABAMA,AL,1001,Autauga,1001020300,2.065365,3476,433,0.547700
8,1001,Autauga,AL,55869,45,9,23,23,45804,1,ALABAMA,AL,1001,Autauga,1001020900,113.034040,6401,532,0.179727
9,1001,Autauga,AL,55869,45,9,23,23,45803,1,ALABAMA,AL,1001,Autauga,1001020700,8.652778,4029,545,0.886712


In [1753]:
#df.FIPS_y.apply(str)
demand_point = df.FIPS_y.tolist()
population_census = df.E_TOTPOP.tolist()
demand_amount = df.Final_Need.tolist()
State_county = df.State.tolist()
CT_location = df.FIPS_x.tolist()
weight_list = df.RPL_ThemesStates.tolist()


In [1754]:
population_county = df.groupby(['FIPS_x'])['E_TOTPOP'].sum().to_dict()
demand_per_county = dict(zip(df.FIPS_x, df.Final_Need))
capacity_county = dict(zip(df.FIPS_x, df.County_Baseline_CT_Need))
print (capacity_county)
#print (demand_per_county)
#print (population_county)
#print (CT_location)
#print (demand_point)
FIPS_County_Name_dict = dict(zip(df.FIPS_y, df.COUNTY))
county_of_states = dict(zip(df.FIPS_x, df.State))
county_name = dict(zip(df.FIPS_x, df.COUNTY))
census_county = dict(zip(df.FIPS_y, df.FIPS_x))
#print (county_of_states)
#print(county_name)

{1001: 9, 1003: 34, 1005: 4, 1007: 4, 1009: 9, 1011: 2, 1013: 3, 1015: 18, 1017: 5, 1019: 4, 1021: 7, 1023: 2, 1025: 4, 1027: 2, 1029: 3, 1031: 8, 1033: 9, 1035: 2, 1037: 2, 1039: 6, 1041: 3, 1043: 13, 1045: 8, 1047: 6, 1049: 11, 1051: 13, 1053: 6, 1055: 16, 1057: 3, 1059: 5, 1061: 4, 1063: 2, 1065: 3, 1067: 3, 1069: 16, 1071: 8, 1073: 99, 1075: 3, 1077: 14, 1079: 5, 1081: 25, 1083: 15, 1085: 2, 1087: 3, 1089: 56, 1091: 3, 1093: 5, 1095: 15, 1097: 62, 1099: 4, 1101: 34, 1103: 18, 1105: 2, 1107: 3, 1109: 5, 1111: 4, 1113: 9, 1115: 14, 1117: 33, 1119: 2, 1121: 12, 1123: 7, 1125: 32, 1127: 10, 1129: 3, 1131: 2, 1133: 4, 2013: 1, 2016: 1, 2020: 44, 2050: 3, 2060: 1, 2068: 1, 2070: 1, 2090: 15, 2100: 1, 2105: 1, 2110: 5, 2122: 9, 2130: 3, 2150: 2, 2158: 2, 2164: 1, 2170: 17, 2180: 2, 2185: 2, 2188: 2, 2195: 1, 2198: 1, 2220: 2, 2230: 1, 2240: 2, 2261: 2, 2275: 1, 2282: 1, 2290: 1, 4001: 11, 4003: 19, 4005: 22, 4007: 9, 4009: 6, 4011: 2, 4012: 4, 4013: 673, 4015: 32, 4017: 17, 4019: 158, 402

In [1755]:
coverage_loc_set = {}
demand = {}
weight = {}
population = {}
county_pop = {}
count = 0
county_demand = {}


for d in demand_point:
    county_demand[census_county[d]] = demand_amount[count]
    county_pop[d] = population_county[census_county[d]]
    population[d] = population_census[count]
    coverage_loc_set[d] = CT_location[count]
    if county_pop[d] >= 1e-6:
        demand[d] = (demand_amount[count]*population[d])/county_pop[d]
    else:
        demand[d] = 0
    #print (d, demand[d], np.floor(demand[d]), np.ceil(demand[d]), demand_amount[count], population[d], county_pop[d])
    weight[d] = weight_list[count]  
    count += 1     
    

    #print (d, demand[d],demand_amount[count])
print (sum (demand_amount)) 
print (sum(demand))
print (county_demand)

51857020
2027110122826345
{1001: 23, 1003: 34, 1005: 12, 1007: 4, 1009: 9, 1011: 8, 1013: 72, 1015: 18, 1017: 14, 1019: 6, 1021: 8, 1023: 13, 1025: 17, 1027: 4, 1029: 3, 1031: 21, 1033: 22, 1035: 3, 1037: 2, 1039: 10, 1041: 15, 1043: 13, 1045: 10, 1047: 43, 1049: 43, 1051: 34, 1053: 6, 1055: 33, 1057: 3, 1059: 94, 1061: 4, 1063: 12, 1065: 14, 1067: 4, 1069: 16, 1071: 8, 1073: 124, 1075: 3, 1077: 29, 1079: 8, 1081: 25, 1083: 15, 1085: 26, 1087: 7, 1089: 56, 1091: 18, 1093: 9, 1095: 130, 1097: 221, 1099: 4, 1101: 164, 1103: 18, 1105: 5, 1107: 11, 1109: 17, 1111: 24, 1113: 13, 1115: 14, 1117: 33, 1119: 23, 1121: 12, 1123: 24, 1125: 41, 1127: 10, 1129: 12, 1131: 12, 1133: 6, 2013: 1, 2016: 1, 2020: 44, 2050: 3, 2060: 1, 2068: 1, 2070: 1, 2090: 15, 2100: 1, 2105: 1, 2110: 5, 2122: 9, 2130: 3, 2150: 2, 2158: 2, 2164: 1, 2170: 17, 2180: 2, 2185: 2, 2188: 2, 2195: 1, 2198: 1, 2220: 2, 2230: 1, 2240: 2, 2261: 2, 2275: 1, 2282: 1, 2290: 1, 4001: 155, 4003: 19, 4005: 130, 4007: 9, 4009: 6, 4011: 

In [1756]:
#others = list(df.columns)
#others.remove('FIPS')

In [1757]:
#demand_point, demand = gp.multidict({demand_p:demand} for demand in demand_county for demand_p in demand_point_x)

new_df = df.drop_duplicates(subset=['FIPS_x'])
capacity = new_df.groupby(['State'])['County_Baseline_CT_Need'].sum().to_dict()

demand_per_State = new_df.groupby(['State'])['Final_Need'].sum().to_dict()
population_per_state =  new_df.groupby(['State'])['Population'].sum().to_dict()
#print (demand_per_State)
not_enough = 0
USA_capacity = 0
for i in capacity:
    USA_capacity += capacity[i]
    if capacity[i] != demand_per_State[i]:
        not_enough += 1
        print (i, capacity[i], demand_per_State[i], (population_per_state[i]/100000)*15)
#print (not_enough)
#print (USA_capacity)

AL 762 1769 735.47775
AR 487 753 452.6706
AZ 1099 2069 1091.80755
CA 5954 11736 5926.83345
CO 898 2670 863.8104
CT 539 3398 534.79305
DC 106 1087 105.86234999999999
DE 148 1019 146.0646
FL 3258 4769 3221.66055
GA 1671 3863 1592.6134499999998
IA 520 2818 473.2605
ID 291 347 268.05975
IL 1950 14275 1900.77315
IN 1055 3805 1009.8328500000001
KS 491 1791 436.9971
KY 727 1378 670.15095
LA 733 2072 697.3191
MA 1040 9210 1033.87545
MD 918 6100 906.852
ME 210 291 201.6318
MI 1539 3457 1498.02855
MN 889 3709 845.9448
MO 974 1631 920.6142
MS 491 1639 446.42235
NC 1626 2902 1573.2126
ND 145 304 114.3093
NE 345 2251 290.1612
NH 209 557 203.95665000000002
NJ 1341 12254 1332.3285
NM 330 1028 314.52434999999997
NV 470 615 462.0234
NY 3577 20627 3544.20975
OH 1801 3978 1753.365
OK 633 847 593.54565
OR 654 741 632.6605500000001
PA 1955 7484 1920.29835
RI 161 1756 158.90415
SC 795 1198 772.3071
SD 167 762 132.69885000000002
TN 1073 2664 1024.3761
TX 4485 7689 4349.38215
UT 498 1022 480.89369999999997
VA

In [1758]:
#Parameters
Budget = 300000
#print (Budget)
#demand_point, weight, demand, coverage_loc_set = gp.multidict({0:[8,5,{0}], 1:[7,1,{0}], 2:[5,10,{0}] ,3:[6,1,{0}] ,4:[10,3,{0}] ,5:[5,2,{0}], 6:[7,6,{0}], 7:[5,4,{0}]})
#CT_location, capacity = gp.multidict({0: 10})
#for i in demand_point:
 #   print (i,coverage_loc_set[i])
location = list(dict.fromkeys(CT_location))
pro = [(i,coverage_loc_set[i]) for i in demand_point]
#print (location)
#print (pro)
pro_c_s = [(i,county_of_states[i]) for i in location]
print (pro_c_s)

[(1001, 'AL'), (1003, 'AL'), (1005, 'AL'), (1007, 'AL'), (1009, 'AL'), (1011, 'AL'), (1013, 'AL'), (1015, 'AL'), (1017, 'AL'), (1019, 'AL'), (1021, 'AL'), (1023, 'AL'), (1025, 'AL'), (1027, 'AL'), (1029, 'AL'), (1031, 'AL'), (1033, 'AL'), (1035, 'AL'), (1037, 'AL'), (1039, 'AL'), (1041, 'AL'), (1043, 'AL'), (1045, 'AL'), (1047, 'AL'), (1049, 'AL'), (1051, 'AL'), (1053, 'AL'), (1055, 'AL'), (1057, 'AL'), (1059, 'AL'), (1061, 'AL'), (1063, 'AL'), (1065, 'AL'), (1067, 'AL'), (1069, 'AL'), (1071, 'AL'), (1073, 'AL'), (1075, 'AL'), (1077, 'AL'), (1079, 'AL'), (1081, 'AL'), (1083, 'AL'), (1085, 'AL'), (1087, 'AL'), (1089, 'AL'), (1091, 'AL'), (1093, 'AL'), (1095, 'AL'), (1097, 'AL'), (1099, 'AL'), (1101, 'AL'), (1103, 'AL'), (1105, 'AL'), (1107, 'AL'), (1109, 'AL'), (1111, 'AL'), (1113, 'AL'), (1115, 'AL'), (1117, 'AL'), (1119, 'AL'), (1121, 'AL'), (1123, 'AL'), (1125, 'AL'), (1127, 'AL'), (1129, 'AL'), (1131, 'AL'), (1133, 'AL'), (2013, 'AK'), (2016, 'AK'), (2020, 'AK'), (2050, 'AK'), (2060

In [1759]:

cartesian_prod = gp.tuplelist(pro)
cartesian_pro_county_state = gp.tuplelist(pro_c_s)
#print (cartesian_prod)


#for (i,j) in cartesian_prod:
 #   if j not in coverage_loc_set[i]:
  #      cartesian_prod.remove((i,j))
       # print (i,j)
   # if j in coverage_loc_set[i]:
    #    print (i,j)
print (cartesian_pro_county_state)

<gurobi.tuplelist (3142 tuples, 2 values each):
 ( 1001  , AL )
 ( 1003  , AL )
 ( 1005  , AL )
 ( 1007  , AL )
 ( 1009  , AL )
 ( 1011  , AL )
 ( 1013  , AL )
 ( 1015  , AL )
 ( 1017  , AL )
 ( 1019  , AL )
 ( 1021  , AL )
 ( 1023  , AL )
 ( 1025  , AL )
 ( 1027  , AL )
 ( 1029  , AL )
 ( 1031  , AL )
 ( 1033  , AL )
 ( 1035  , AL )
 ( 1037  , AL )
 ( 1039  , AL )
 ( 1041  , AL )
 ( 1043  , AL )
 ( 1045  , AL )
 ( 1047  , AL )
 ( 1049  , AL )
 ( 1051  , AL )
 ( 1053  , AL )
 ( 1055  , AL )
 ( 1057  , AL )
 ( 1059  , AL )
 ( 1061  , AL )
 ( 1063  , AL )
 ( 1065  , AL )
 ( 1067  , AL )
 ( 1069  , AL )
 ( 1071  , AL )
 ( 1073  , AL )
 ( 1075  , AL )
 ( 1077  , AL )
 ( 1079  , AL )
 ( 1081  , AL )
 ( 1083  , AL )
 ( 1085  , AL )
 ( 1087  , AL )
 ( 1089  , AL )
 ( 1091  , AL )
 ( 1093  , AL )
 ( 1095  , AL )
 ( 1097  , AL )
 ( 1099  , AL )
 ( 1101  , AL )
 ( 1103  , AL )
 ( 1105  , AL )
 ( 1107  , AL )
 ( 1109  , AL )
 ( 1111  , AL )
 ( 1113  , AL )
 ( 1115  , AL )
 ( 1117  , AL )
 ( 1119 

In [1760]:
#MIP model formulation
m = gp.Model("Contact_Tracing_Coverage")

In [1761]:

#Add variable for each contact tracer center
y = m.addVars(location, vtype = GRB.INTEGER, name = "y")


#Add variable for each demand point (census tract/block)
x = m.addVars(cartesian_prod, vtype = GRB.CONTINUOUS, name = "x")
m.update()


In [1762]:
#for j in CT_location:
    #print (j)
#for i in demand_point:
    #print (i)
    #for j in CT_location:
        #print (j)
        #if j in coverage_loc_set[i]:
            #print (j, 'can cover', i)

In [1763]:
#Coverage constraint

m.addConstrs(gp.quicksum(x[i,j] for (i,j) in cartesian_prod.select(i,'*')) <= demand[i] for i in demand_point)

{1001020200: <gurobi.Constr *Awaiting Model Update*>,
 1001021000: <gurobi.Constr *Awaiting Model Update*>,
 1001020100: <gurobi.Constr *Awaiting Model Update*>,
 1001020801: <gurobi.Constr *Awaiting Model Update*>,
 1001021100: <gurobi.Constr *Awaiting Model Update*>,
 1001020400: <gurobi.Constr *Awaiting Model Update*>,
 1001020600: <gurobi.Constr *Awaiting Model Update*>,
 1001020300: <gurobi.Constr *Awaiting Model Update*>,
 1001020900: <gurobi.Constr *Awaiting Model Update*>,
 1001020700: <gurobi.Constr *Awaiting Model Update*>,
 1001020500: <gurobi.Constr *Awaiting Model Update*>,
 1001020802: <gurobi.Constr *Awaiting Model Update*>,
 1003011408: <gurobi.Constr *Awaiting Model Update*>,
 1003011405: <gurobi.Constr *Awaiting Model Update*>,
 1003011406: <gurobi.Constr *Awaiting Model Update*>,
 1003011102: <gurobi.Constr *Awaiting Model Update*>,
 1003011300: <gurobi.Constr *Awaiting Model Update*>,
 1003011202: <gurobi.Constr *Awaiting Model Update*>,
 1003010903: <gurobi.Constr 

In [1764]:
#m.addConstrs(z[i] <= gp.quicksum(x[i,j] for (i,j) in cartesian_prod.select(i,'*')) for i in demand_point)

In [1765]:
#Each demand point can be covered at most one location
m.addConstrs(gp.quicksum(x[i,j] for (i,j) in cartesian_prod.select('*',j)) <=   y[j] for j in location)

{1001: <gurobi.Constr *Awaiting Model Update*>,
 1003: <gurobi.Constr *Awaiting Model Update*>,
 1005: <gurobi.Constr *Awaiting Model Update*>,
 1007: <gurobi.Constr *Awaiting Model Update*>,
 1009: <gurobi.Constr *Awaiting Model Update*>,
 1011: <gurobi.Constr *Awaiting Model Update*>,
 1013: <gurobi.Constr *Awaiting Model Update*>,
 1015: <gurobi.Constr *Awaiting Model Update*>,
 1017: <gurobi.Constr *Awaiting Model Update*>,
 1019: <gurobi.Constr *Awaiting Model Update*>,
 1021: <gurobi.Constr *Awaiting Model Update*>,
 1023: <gurobi.Constr *Awaiting Model Update*>,
 1025: <gurobi.Constr *Awaiting Model Update*>,
 1027: <gurobi.Constr *Awaiting Model Update*>,
 1029: <gurobi.Constr *Awaiting Model Update*>,
 1031: <gurobi.Constr *Awaiting Model Update*>,
 1033: <gurobi.Constr *Awaiting Model Update*>,
 1035: <gurobi.Constr *Awaiting Model Update*>,
 1037: <gurobi.Constr *Awaiting Model Update*>,
 1039: <gurobi.Constr *Awaiting Model Update*>,
 1041: <gurobi.Constr *Awaiting Model Up

In [1766]:
#for i in demand_point :
 #   print (i,demand[i])
#total_cap = 0
#for j in CT_location:
 #   total_cap += capacity[j]
  #  print (j, capacity[j]/demand_per_State[j])

In [1767]:
#print (total_cap)
epsilon = 0.3

In [1768]:
#for i in demand_point:
 #   print (i, FIPS_County_Name_dict[i], demand[i]/population[i], 'demand', demand[i], population[i])

In [1769]:
#for j in CT_location:
    #print (j, capacity[j]/demand_per_State[j], demand_per_State[j], capacity[j] )

In [1770]:
#m.addConstrs(gp.quicksum((x[i,j]/demand[i]) for (i,j) in cartesian_prod.select(i,'*') ) >= -0.003+ min( capacity[j]/demand_per_State[j] for j in CT_location) for i in demand_point)

In [1771]:
#for (j,s) in cartesian_pro_county_state:
 #   print (j,s, capacity_State[s], y[j])


In [1772]:

for s in capacity:
    print (s,capacity[s])
    m.addConstr(gp.quicksum(y[j] for j in location if (j,s) in cartesian_pro_county_state) <= (30/15)*capacity[s])

AK 126
AL 762
AR 487
AZ 1099
CA 5954
CO 898
CT 539
DC 106
DE 148
FL 3258
GA 1671
HI 216
IA 520
ID 291
IL 1950
IN 1055
KS 491
KY 727
LA 733
MA 1040
MD 918
ME 210
MI 1539
MN 889
MO 974
MS 491
MT 191
NC 1626
ND 145
NE 345
NH 209
NJ 1341
NM 330
NV 470
NY 3577
OH 1801
OK 633
OR 654
PA 1955
RI 161
SC 795
SD 167
TN 1073
TX 4485
UT 498
VA 1347
VT 101
WA 1162
WI 911
WV 301
WY 98


In [1773]:
m.addConstrs(y[j] <=   demand_per_county[j] for j in location)

{1001: <gurobi.Constr *Awaiting Model Update*>,
 1003: <gurobi.Constr *Awaiting Model Update*>,
 1005: <gurobi.Constr *Awaiting Model Update*>,
 1007: <gurobi.Constr *Awaiting Model Update*>,
 1009: <gurobi.Constr *Awaiting Model Update*>,
 1011: <gurobi.Constr *Awaiting Model Update*>,
 1013: <gurobi.Constr *Awaiting Model Update*>,
 1015: <gurobi.Constr *Awaiting Model Update*>,
 1017: <gurobi.Constr *Awaiting Model Update*>,
 1019: <gurobi.Constr *Awaiting Model Update*>,
 1021: <gurobi.Constr *Awaiting Model Update*>,
 1023: <gurobi.Constr *Awaiting Model Update*>,
 1025: <gurobi.Constr *Awaiting Model Update*>,
 1027: <gurobi.Constr *Awaiting Model Update*>,
 1029: <gurobi.Constr *Awaiting Model Update*>,
 1031: <gurobi.Constr *Awaiting Model Update*>,
 1033: <gurobi.Constr *Awaiting Model Update*>,
 1035: <gurobi.Constr *Awaiting Model Update*>,
 1037: <gurobi.Constr *Awaiting Model Update*>,
 1039: <gurobi.Constr *Awaiting Model Update*>,
 1041: <gurobi.Constr *Awaiting Model Up

In [1774]:
#scalar = 0.00005
#for i in demand_point :
#    print (i, FIPS_County_Name_dict[i], weight[i], demand[i]/sum(demand_per_State[j] for (i,j) in cartesian_prod.select(i,'*')))

In [1775]:
hot_stop_weight = {} #demand divided by pop if no pop then it is 0
total = 0
for j in location:
    
    if population_county[j] <1e-6:
        hot_stop_weight[j] = 0
    else:
        hot_stop_weight[j] = demand_per_county[j]/population_county[j]
        total += hot_stop_weight[j]
        #print(j, county_demand[j], demand_per_county[j], population_county[j])

max_hot = max(hot_stop_weight[j] for j in location)
min_hot = min(hot_stop_weight[j] for j in location)
divide_multip = 1/(max_hot-min_hot)
print (total) 
print (max(hot_stop_weight[j] for j in location))
print (min(hot_stop_weight[j] for j in location))

1.442969006564771
0.045649221769560225
0.0001432870038687491


In [1776]:
m.setObjective(gp.quicksum(weight[i]*population[i]*x[i,j] for (i,j) in cartesian_prod ) + gp.quicksum((hot_stop_weight[j]*100000)*y[j] for j in location), GRB.MAXIMIZE)

In [1777]:
m.update()
m.write("CT_coverage_model.lp")
#m.computeIIS()
#m.write("model.ilp")

In [1778]:
m.update()
m.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 79172 rows, 75979 columns and 155100 nonzeros
Model fingerprint: 0x4aa2a7bb
Variable types: 72837 continuous, 3142 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-03, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-04, 1e+04]
Found heuristic solution: objective 2.634487e+08
Presolve removed 79110 rows and 71100 columns
Presolve time: 0.47s
Presolved: 62 rows, 4879 columns, 4940 nonzeros
Found heuristic solution: objective 3.356481e+08
Variable types: 4817 continuous, 62 integer (1 binary)

Root relaxation: objective 3.496963e+08, 64 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3.4970e+08    0   53 3.3565e+08 3.4970e+08  4.19%     -    0s
H    0     0                    3.496835e+08 3.4970e+08  0.00%  

In [1779]:

#print (m.display())

In [1780]:
covered_or_not = []
demand_c = []
location_c = []
percentage = []
percentage_d = {}
county_name_t = []
state_name = []

for i in demand_point:
    if (sum(abs(x[i,j].x) for (i,j) in cartesian_prod.select(i,'*')) < 1e-6 ):
        #print (f"\n Demand point {FIPS_County_Name_dict[i]} with FIPS {i} with total demand {demand[i]} and SVI {weight[i]} is NOT covered by contact tracer location {coverage_loc_set [i]}, {county_of_states[FIPS_County_Name_dict[i]]}.")
        covered_or_not.append(0)
        demand_c.append(str(i).zfill(11))
        county_name_t.append(FIPS_County_Name_dict[i])
        state_name.append(county_of_states[census_county[i]])
        if demand[i] > 1e-6:
            percentage.append(100*sum(abs(x[i,j].x) for (i,j) in cartesian_prod.select(i,'*'))/demand[i])
            percentage_d [i] = 100*sum(abs(x[i,j].x) for (i,j) in cartesian_prod.select(i,'*'))/demand[i]
            
        else:
            percentage.append(0)
            percentage_d [i] = 0
            
        #print (i,0)
    elif (sum(abs(x[i,j].x) for (i,j) in cartesian_prod.select(i,'*')) > 1e-6 ):
        #print (f"\n Demand point {FIPS_County_Name_dict[i]} with FIPS {i} with total demand {demand[i]} and SVI {weight[i]}: covered demand {sum(abs(x[i,j].x) for (i,j) in cartesian_prod.select(i,'*'))} by contact tracer location {coverage_loc_set [i]}, {county_of_states[FIPS_County_Name_dict[i]]}.")
        county_name_t.append(FIPS_County_Name_dict[i])
        state_name.append(county_of_states[census_county[i]])        
        covered_or_not.append(1)
        demand_c.append(str(i).zfill(11))
        if demand[i] > 1e-6:
            percentage.append(100*sum(abs(x[i,j].x) for (i,j) in cartesian_prod.select(i,'*'))/demand[i])
            percentage_d [i] = 100*sum(abs(x[i,j].x) for (i,j) in cartesian_prod.select(i,'*'))/demand[i]
        else:
            percentage.append(0)
            percentage_d [i] = 0
        #print (i,1)



demand_covered_per_state = {}
percentage_covered_per_state = {}
capacity_enough = {}
county_flow = []
county_percentage = []
county_name_c = []
state_name_c = []
demand_of_county = []
hot_spot = []
for j in location:
    location_c.append(str(j).zfill(5))
    hot_spot.append(100*hot_stop_weight[j])
    
    county_flow.append(y[j].x)
    demand_of_county.append(demand_per_county[j])
    county_percentage.append((y[j].x/demand_per_county[j])*100)
    county_name_c.append(county_name[j])
    state_name_c.append(county_of_states[j])
    #print (str(j).zfill(5), y[j].x, demand_per_county[j], capacity_county[j], county_name[j], county_of_states[j])
    
for j in capacity:
    if capacity[j] != demand_per_State[j]:
        capacity_enough[j] = 0
    else :
        capacity_enough[j] = 1
        
#for j in CT_location:
#    demand_covered_per_state[j] = sum(abs(x[i,j].x) for (i,j) in cartesian_prod.select('*',j))
#    percentage_covered_per_state[j] = (demand_covered_per_state[j]/demand_per_State[j])*100

#    print (f"\n State {j} can cover  {round(percentage_covered_per_state[j],2)} % of demand and capacity situation is {capacity_enough[j]}")
print (location_c)   

['01001', '01003', '01005', '01007', '01009', '01011', '01013', '01015', '01017', '01019', '01021', '01023', '01025', '01027', '01029', '01031', '01033', '01035', '01037', '01039', '01041', '01043', '01045', '01047', '01049', '01051', '01053', '01055', '01057', '01059', '01061', '01063', '01065', '01067', '01069', '01071', '01073', '01075', '01077', '01079', '01081', '01083', '01085', '01087', '01089', '01091', '01093', '01095', '01097', '01099', '01101', '01103', '01105', '01107', '01109', '01111', '01113', '01115', '01117', '01119', '01121', '01123', '01125', '01127', '01129', '01131', '01133', '02013', '02016', '02020', '02050', '02060', '02068', '02070', '02090', '02100', '02105', '02110', '02122', '02130', '02150', '02158', '02164', '02170', '02180', '02185', '02188', '02195', '02198', '02220', '02230', '02240', '02261', '02275', '02282', '02290', '04001', '04003', '04005', '04007', '04009', '04011', '04012', '04013', '04015', '04017', '04019', '04021', '04023', '04025', '04027', 

In [1781]:
#for (i,j) in cartesian_prod:
 #   if (abs(x[i,j].x) > 1e-6 ):
  #      print (f"\n Demand point {i} with {x[i,j].x} of total demand {demand[i]} with percentage {percentage_d[i]} is covered by contact tracer center {j}, {county_of_states[j]}.")

In [1782]:
import csv

writefile = '../Contact_Tracer_partial_demand_and_coverage_USA_county_census.csv'
fieldnames = ['Census Tract FIPS', 'demand_covered','percentage', 'county_name', 'state_name']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)
    writer.writerow(fieldnames)
    for row in zip(demand_c, covered_or_not, percentage, county_name_t, state_name):
        #print (row)
        writer.writerow(row)

In [1783]:
#import geopandas as gpd
#from census import Census
#import matplotlib.pyplot as plt
#import contextily as ctx
#pd.options.display.max_columns =200
#c = Census("d95e144b39e17f929287714b0b8ba9768cecdc9f")

In [1784]:
writefile = '../Contact_Tracer_partial_demand_and_coverage_USA_county_COVERAGE_census.csv'
fieldnames = ['County FIPS', 'demand_covered','total_demand_of_county','percentage', 'hot_spot', 'county_name', 'state_name']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)
    writer.writerow(fieldnames)
    for row in zip(location_c, county_flow, demand_of_county, county_percentage, hot_spot, county_name_c, state_name_c):
        print (row)
        writer.writerow(row)

('01001', 19.0, 23, 82.6086956521739, 0.04166666666666667, 'Autauga', 'AL')
('01003', 29.0, 34, 85.29411764705883, 0.01633774933087306, 'Baldwin', 'AL')
('01005', 12.0, 12, 100.0, 0.046544100535257156, 'Barbour', 'AL')
('01007', 3.0, 4, 75.0, 0.017756470013761263, 'Bibb', 'AL')
('01009', 9.0, 9, 100.0, 0.0156128024980484, 'Blount', 'AL')
('01011', 8.0, 8, 100.0, 0.07727975270479134, 'Bullock', 'AL')
('01013', 61.0, 72, 84.72222222222221, 0.3595505617977528, 'Butler', 'AL')
('01015', 17.0, 18, 94.44444444444444, 0.01563884689568889, 'Calhoun', 'AL')
('01017', 14.0, 14, 100.0, 0.04138828120380772, 'Chambers', 'AL')
('01019', 6.0, 6, 100.0, 0.02320813832050439, 'Cherokee', 'AL')
('01021', 8.0, 8, 100.0, 0.01821078989301161, 'Chilton', 'AL')
('01023', 13.0, 13, 100.0, 0.09942638623326959, 'Choctaw', 'AL')
('01025', 15.0, 17, 88.23529411764706, 0.0697092713330873, 'Clarke', 'AL')
('01027', 3.0, 4, 75.0, 0.02989983555090447, 'Clay', 'AL')
('01029', 3.0, 3, 100.0, 0.020083009773731425, 'Clebu

('19187', 2.0, 6, 33.33333333333333, 0.016323421389123162, 'Webster', 'IA')
('19189', 0.0, 2, 0.0, 0.01891968593321351, 'Winnebago', 'IA')
('19191', 0.0, 3, 0.0, 0.014705161511690604, 'Winneshiek', 'IA')
('19193', 235.0, 480, 48.95833333333333, 0.4687591554522549, 'Woodbury', 'IA')
('19195', 0.0, 2, 0.0, 0.026705835224996664, 'Worth', 'IA')
('19197', 0.0, 2, 0.0, 0.015620118712902217, 'Wright', 'IA')
('20001', 0.0, 2, 0.0, 0.015835312747426764, 'Allen', 'KS')
('20003', 1.0, 2, 50.0, 0.025471217524197655, 'Anderson', 'KS')
('20005', 1.0, 3, 33.33333333333333, 0.01833404632402371, 'Atchison', 'KS')
('20007', 0.0, 1, 0.0, 0.021128248468201986, 'Barber', 'KS')
('20009', 2.0, 8, 25.0, 0.029860774140569594, 'Barton', 'KS')
('20011', 0.0, 3, 0.0, 0.020405387022173854, 'Bourbon', 'KS')
('20013', 0.0, 2, 0.0, 0.020695364238410598, 'Brown', 'KS')
('20015', 3.0, 11, 27.27272727272727, 0.016549316964554372, 'Butler', 'KS')
('20017', 0.0, 1, 0.0, 0.03780718336483932, 'Chase', 'KS')
('20019', 0.0, 1

('42081', 16.0, 35, 45.714285714285715, 0.030472144107122647, 'Lycoming', 'PA')
('42083', 5.0, 7, 71.42857142857143, 0.01674400803712386, 'McKean', 'PA')
('42085', 10.0, 17, 58.82352941176471, 0.015093669537423422, 'Mercer', 'PA')
('42087', 6.0, 10, 60.0, 0.021569388723523576, 'Mifflin', 'PA')
('42089', 42.0, 56, 75.0, 0.03341567911400714, 'Monroe', 'PA')
('42091', 147.0, 602, 24.418604651162788, 0.07329834007264084, 'Montgomery', 'PA')
('42093', 1.0, 3, 33.33333333333333, 0.016398819285011478, 'Montour', 'PA')
('42095', 76.0, 242, 31.40495867768595, 0.08019139897540575, 'Northampton', 'PA')
('42097', 8.0, 17, 47.05882352941176, 0.018413214189006228, 'Northumberland', 'PA')
('42099', 4.0, 7, 57.14285714285714, 0.015242574688615975, 'Perry', 'PA')
('42101', 1755.0, 2233, 78.59381997313032, 0.14173080413983427, 'Philadelphia', 'PA')
('42103', 8.0, 32, 25.0, 0.05765973548596346, 'Pike', 'PA')
('42105', 2.0, 3, 66.66666666666666, 0.017712700005904233, 'Potter', 'PA')
('42107', 25.0, 53, 47

In [1785]:
#shape_file = gpd.read_file('SVI2018_US_COUNTY.zip')